In [ ]:
# Useful imports
import numpy as np, sys; sys.path.append('..')
from matplotlib import pyplot as plt
import corner, emcee
from autocorr_time import integrated_time

------------------
# Code to change between runs

In [ ]:
# Load the samples file
file_name = 'samples.npz'

# Specify the burn-in and n_steps
n_burn = 100
n_steps = 300

# End code to change
-----------------

### Get an array of the samples

This block performs analysis on the sets of samples to determine if walkers were stuck, thin by the autocorrelation time, and reshapes if necessary.

In [ ]:
# Determine the sampling method (.h5 -> emcee, .npz -> ptemcee)
if file_name.endswith('.h5'):
# Load in the samples
    reader = emcee.backends.HDFBackend(file_name)
    samples_not_flat = reader.get_chain()
    samples_flat = reader.get_chain(flat = True)

    # Look at the variances within each chain to determine if the walker is moving enough or if it is stuck.
    within_chain_means = np.mean(samples_not_flat[n_burn:, :, :], axis = 0)

    # Create an empty array of the within chain variances
    within_chain_var = np.empty(within_chain_means.shape)

    # Run a for loop across all walkers to compute the within chain variance
    for i in range(0, within_chain_means.shape[0]):
        within_chain_var[i, :] = np.sum(np.square(within_chain_means[i, :] - samples_not_flat[n_burn:, i, :]), axis = 0) / (samples_not_flat.shape[0] // 2)

    # Get the typical within chain variance W for each parameter
    W = np.median(within_chain_var, axis = 0)

    # Now we need to loop over each chain for each parameter to see how it compares to the typical variance
    bad_indices = []
    ratios = np.empty(within_chain_means.shape)
    # Loop over each parameter
    for i in range(0, within_chain_means.shape[1]):
        # Loop over the walkers
        for j in range(0, within_chain_means.shape[0]):
            ratio = np.sum(within_chain_var[j, i] / W[i]) / within_chain_means.shape[1]
            ratios[j, i] = ratio

    # Sum along each parameter, this value should be very close to 1.0. Select out the bad indices
    total_normalized_ratios = np.sum(ratios, axis = 1)
    bad_indices = np.where(total_normalized_ratios <= 0.9)[0]
    print('Found {} bad walkers at indices:'.format(bad_indices.shape[0]))
    print(bad_indices)

    if bad_indices.shape[0] != 0:
        # Remove the bad walkers
        samples_not_flat = np.delete(samples_not_flat, bad_indices, axis = 1)

    # # Thin according to the burn-in time
    thinned_samples_not_flat = samples_not_flat[n_burn:, :, :]

    # Compute the autocorrelation times for each parameter
    ac_s = reader.get_autocorr_time(discard = n_burn, tol = 0)
    ac = int(np.max(ac_s))
    print('Autocorrelation time: {}'.format(ac))

    # Thin according to the autocorrelation time
    thinned_samples_not_flat = thinned_samples_not_flat[::ac, :, :]

    # Flatten the samples and log-prob
    len0, len1, len2 = thinned_samples_not_flat.shape
    samples = np.reshape(thinned_samples_not_flat, (len0 * len1, len2))
elif file_name.endswith('.npz'):
    # Load in the samples
    all_samples = np.load(file_name)['arr_0']

    samples_not_flat = all_samples[0] # Just the beta = 1 samples
    # Swap axes so it is in the shape (step, walker, parameter)
    samples_not_flat = np.swapaxes(samples_not_flat, 0, 1)

    # Look at the variances within each chain to determine if the walker is moving enough or if it is stuck.
    within_chain_means = np.mean(samples_not_flat[:, :, :], axis = 0)

    # Create an empty array of the within chain variances
    within_chain_var = np.empty(within_chain_means.shape)

    # Run a for loop across all walkers to compute the within chain variance
    for i in range(0, within_chain_means.shape[0]):
        within_chain_var[i, :] = np.sum(np.square(within_chain_means[i, :] - samples_not_flat[:, i, :]), axis = 0) / (samples_not_flat.shape[0] // 2)

    # Get the typical within chain variance W for each parameter
    W = np.median(within_chain_var, axis = 0)


    # Now we need to loop over each chain for each parameter to see how it compares to the typical variance
    bad_indices = []
    ratios = np.empty(within_chain_means.shape)
    # Loop over each parameter
    for i in range(0, within_chain_means.shape[1]):
        # Loop over the walkers
        for j in range(0, within_chain_means.shape[0]):
            ratio = np.sum(within_chain_var[j, i] / W[i]) / within_chain_means.shape[1]
            ratios[j, i] = ratio

    # Sum along each parameter, this value should be very close to 1.0. Select out the bad indices
    total_normalized_ratios = np.sum(ratios, axis = 1)
    bad_indices = np.where(total_normalized_ratios <= 0.9)[0]
    print('Found {} bad walkers at indices:'.format(bad_indices.shape[0]))
    print(bad_indices)

    if bad_indices.shape[0] != 0:
        # Remove the bad walkers
        samples_not_flat = np.delete(samples_not_flat, bad_indices, axis = 1)

    # Compute the autocorrelation time and thin
    ac_s = integrated_time(samples_not_flat)
    ac = int(np.ceil(np.max(ac_s)))
    samples_not_flat = samples_not_flat[::ac, :, :]
    print('Autocorrelation time: {}'.format(ac))

    # Flatten the samples
    len0, len1, len2 = samples_not_flat.shape
    samples = np.reshape(samples_not_flat, (len0 * len1, len2))

In [ ]:
# Generate a corner plot of the samples
labels = ['$A_{0}$', '$r_{0}$', '$(C_{1}^{+})^{2}$', '$P_{1}^{+}$', '$(C_{1}^{-})^{2}$', '$P_{1}^{-}$']

corner.corner(samples[:, :6], labels = labels, quantiles = [0.16, 0.5, 0.84], show_titles = True, title_fmt = '.3f', title_kwargs = {'fontsize': 12})